In [1]:
# design a sinc RF pulse and simulate it
using BlochSim
using Plots
plotly()
using STFR#: getrf # to design an initial pulse we can simulate
using ForwardDiff
using ProgressMeter
using MAT  # to load designed pulse
using MatrixDepot

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots C:\Users\Stellarlet\.julia\packages\Plots\XuV6v\src\backends.jl:372


LoadError: ArgumentError: Package MatrixDepot not found in current path:
- Run `import Pkg; Pkg.add("MatrixDepot")` to install the MatrixDepot package.


In [2]:
nrf = 128
dt = 0.01 # ms
α = π / 4
vars = matread("test_pulses/slr_tb2.mat")
rf = transpose(vars["rf"])
plot(rf)

<!DOCTYPE html>
 
 
 Plots.jl

In [25]:
#Define Brian Hargreave's Bloch simulator for arbitary phased rf pulses

#define rotations
function xrot(phi)
    return [1 0 0; 0 cos(phi) -sin(phi);0 sin(phi) cos(phi)]
end

function yrot(phi)
    return [cos(phi) 0 sin(phi);0 1 0;-sin(phi) 0 cos(phi)]
end

function zrot(phi)
    return [cos(phi) -sin(phi) 0;sin(phi) cos(phi) 0; 0 0 1]
end

function throt(phi,theta)
    #rotation about axis defined by y = x*tan(theta)
    #yheta = atan(by/bx)
    
    Rz = zrot(-theta);
    Rx = xrot(phi);
    Rth = inv(Rz)*Rx*Rz;
    return Rth
end

function blochsim_B1_arb(α)
    #α= rfp + b1 + m0
    #rfp = rfp_bs + rfp_ss;
    
    nt = length(α)-4;
    b1_0 = α[end-3];
    rfp = α[1:nt];
    
    dt = 1e-6;
    rfp_abs = broadcast(abs, rfp);
    rfp_abs = reshape(rfp_abs,(size(rfp_abs)...,1))
    rfp_abs = 2 .* pi .* 4258 .* dt .* rfp_abs;
    rfp_angle = broadcast(angle, rfp);
    rfp_angle = reshape(rfp_angle,(size(rfp_angle)...,1))

    m0 = α[end-2:end];

    # m = repmat(m0,1, length(b1));  # mx, my, mz for all b1 points
    
    m = m0;
    
    for tt = 1:nt
        rf_b1 = rfp_abs[tt] * b1_0;
        R = throt(rf_b1,rfp_angle[tt]);
        m = R * m;
    end

#for tt = 1:nt
#    rf_b1 = rfp_abs(tt)*b1;
#    for bb = 1:length(b1)
#        R = throt(rf_b1(bb),rfp_angle(tt));
#        m(:,bb) = R*m(:,bb);
#    end

#end

 #   plot(b1,m'), legend('Mx', 'My', 'Mz')
    return m
end

blochsim_B1_arb (generic function with 1 method)

In [26]:
# test the previous BlochSim function
vars = matread("test_pulses/bsse_pulse_raw.mat")

b1 = 0:0.025:3;  # Gauss
nb1 = length(b1);

Mxd = zeros(nb1, 1)
Myd = zeros(nb1, 1)
Mzd = zeros(nb1, 1)
Mx0 = 0
My0 = 0
Mz0 = 1.0

rfp = transpose(vars["rfp_bs"] + vars["rfp_ss"]);

for ii = 1 : nb1
    rfg = [rfp; b1[ii]; Mx0; My0; Mz0];
    M = blochsim_B1(rfg)
    Mxd[ii] = M[1]
    Myd[ii] = M[2]
    Mzd[ii] = M[3]
end

plot(abs.(Complex.(Mxd, Myd)), label="|Mxy|")
plot!(Mxd, label="Mx")
plot!(Myd, label="My")

<!DOCTYPE html>
 
 
 Plots.jl

In [ ]:
# Define my own blochsim that is differentiable
# change the function to include B1 as a variable - HS 6/10
function myBlochSimB1(α)
       
    # α: vector of RF rotations, plus a gradient rotation at the end

    N = length(α) - 5 # number of points in pulse
    b1_input = α[end - 4] # input B1 as an variable

    
    # initialize magnetization
    Mx = α[end - 2]
    My = α[end - 1]
    Mz = α[end]
    
    # apply prewinding gradient rotation
    #cg = cos(-N / 2 * α[end-3])
    #sg = sin(-N / 2 * α[end-3])
    #Mxi = Mx
    #Myi = My
    #Mx = cg * Mxi + sg * Myi
    #My = -sg * Mxi + cg * Myi
    
    # pre-calculate gradient rotation params
    cg = cos(α[end-3])
    sg = sin(α[end-3])
    for ii = 1 : length(α) - 5
        
        # calculate RF rotation params
        crf = cos(α[ii] * b1_input)
        srf = sin(α[ii] * b1_input)
        
        # apply RF rotation
        Myi = My
        Mzi = Mz
        My = crf * Myi + srf * Mzi
        Mz = -srf * Myi + crf * Mzi
        
        # apply gradient rotation
        Mxi = Mx
        Myi = My
        Mx = cg * Mxi + sg * Myi
        My = -sg * Mxi + cg * Myi
            
    end
    
    # apply rewinding gradient rotation
    #cg = cos(-N / 2 * α[end-3])
    #sg = sin(-N/ 2 * α[end-3])
    #Mxi = Mx
    #Myi = My
    #Mx = cg * Mxi + sg * Myi
    #My = -sg * Mxi + cg * Myi
    
    return Mx, My, Mz
    
end

In [ ]:
b1 = 0 : 0.039 : 0.039*127
nb1 = length(b1)

In [ ]:
function B12wrf(b1, wbs)
# B12wrf exact bloch-siegert shift calculator
# b1 - transmit field strength in Gauss. Can be a scalar or a vector of b1's
# wbs - the frequency offset of the bloch-siegert pulse
    wrf = wbs .* (sqrt.((1 .+ (4258 .* b1) .^ 2 ./ wbs .^ 2)) .- 1)
    return wrf
end

In [ ]:
vec_b1 = ones(nrf, 1)
for i = 1:nrf
    vec_b1[i]=b1[i]
end
wrf = B12wrf(vec_b1,5000)
plot(wrf)

In [ ]:
sqrt_b1 = ones(nrf, 1)
for i = 1:nb1
    sqrt_b1[i]=sqrt.(b1[i])
end

In [ ]:
sin_b1 = ones(nrf, 1)
for i = 1:nb1
    sin_b1[i]=sin(b1[i])
end

In [ ]:
# Simulate to check that magnitude is same as BlochSim.jl, and get target pattern to recover the pulse
Mxd = zeros(nb1, 1)
Myd = zeros(nb1, 1)
Mzd = zeros(nb1, 1)
Mx0 = 0
My0 = 0
Mz0 = 1.0
for ii = 1 : nb1
    rfg = [rf * dt / 1000 * GAMMA;b1[ii];  0 * GAMMA * dt / 1000; Mx0; My0; Mz0];
    M = myBlochSimB1(rfg)
    Mxd[ii] = M[1]
    Myd[ii] = M[2]
    Mzd[ii] = M[3]
end
plot(abs.(Complex.(Mxd, Myd)), label="|Mxy|")
plot!(Mxd, label="Mx")
plot!(Myd, label="My")

In [27]:
# define a BlochSim we can use to calculate error - should encapsulate myBlochSim here
#also slightly modified - HS 6/10
function myBlochSimErr(α)
       
    # α: vector of RF rotations, plus a gradient rotation at the end, plus a target vector, plus an error weight
        
    Mxd = α[end-3]
    Myd = α[end-2]
    Mzd = α[end-1]     
    w = α[end]
    
    # Mx, My, Mz = myBlochSimB1(α[1 : end - 4])
    Mx, My, Mz = blochsim_B1_arb(α[1 : end - 4])
    
    err = w * ((Mx - Mxd)^2 + (My - Myd)^2 + (Mz - Mzd)^2)
    # err = w * (Mz - Mzd)^2
    return err
    
end

myBlochSimErr (generic function with 1 method)

In [ ]:
# check that it runs
myBlochSimErr([rf ;b1[floor(Int,nb1/2)] ;0 * GAMMA * dt / 1000; 0; 0; 1.0; 0; 0.61; -0.8; 1])

In [ ]:
step = 0.001
# rfoc = zeros(nrf, 1)
rfoc = sqrt_b1 .* wrf
mega_iters = 5
niters = 100
Mx0 = 0
My0 = 0
Mz0 = 1.0
g = b1 -> ForwardDiff.gradient(myBlochSimErr, b1)
#h = x -> ForwardDiff.hessian(myBlochSimErr, x)

for mnn = 1 : mega_iters
    
@showprogress 1 "Computing..." for nn = 1 : niters
    J = zeros(nrf, 1)
    # J = sqrt_b1
    #H = zeros(nrf, nrf)
    for ii = 1 : nb1
        rfg = [rfoc * dt / 1000 * GAMMA; b1[ii]; Mx0; My0; Mz0; Mxd[ii]; Myd[ii]; Mzd[ii]; 1.0]
        J += g(rfg)[1 : end - 9] 
        #rfg = [rfoc * dt / 1000 * GAMMA; b1[ii]; 0 * GAMMA * dt / 1000; Mx0; My0; Mz0; Mxd[ii]; Myd[ii]; Mzd[ii]; 1.0]
        #    J += g(rfg)[1 : end - 9] 
        #H += h(rfg)[1 : end - 4, 1 : end - 4]
    end
    #rfoc -= (H \ J) ./ (dt / 1000 * GAMMA)
    rfoc -= step * J
end
display(plot(rfoc))
    
end

In [ ]:
plot(rf - rfoc)

In [28]:
# now let's set up our own target patterns
function dinf(d1, d2)

    a1 = 5.309e-3
    a2 = 7.114e-2
    a3 = -4.761e-1
    a4 = -2.66e-3
    a5 = -5.941e-1
    a6 = -4.278e-1

    l10d1 = log10(d1)
    l10d2 = log10(d2)

    d = (a1 * l10d1^2 + a2 * l10d1 + a3) * l10d2 + (a4 * l10d1^2 + a5 * l10d1 + a6)
    
    return d
    
end

dinf (generic function with 1 method)

In [29]:
# do an inversion pulse design
tb = 8
d1 = 0.01
d2 = 0.01
ftw = dinf(d1 / 8.0, sqrt(d2 / 2.0)) / tb # inversion transition width relationship
# set up target pattern
N = 128
f = [0, (1 - ftw) * (tb / 2), (1 + ftw) * (tb / 2), (N / 2)] / (N / 2)
os = 8

x = (-N / 2 : 1 / os : N / 2 - 1 / os)

b1 = (0:0.04:0.04*1023)
println(x)
println(size(x))
nb1 = length(b1)

Mxd = zeros(N * os, 1)
Myd = zeros(N * os, 1)
Mzd = ones(N * os, 1)
Mzd[abs.(collect(x) ./ (N / 2)) .< f[2]] .= -1.0
w = zeros(N * os, 1)
w[abs.(collect(x) ./ (N / 2)) .< f[2]] .= d1 / d2
w[abs.(collect(x) ./ (N / 2)) .> f[3]] .= 1.0

-64.0:0.125:63.875
(1024,)


943-element view(::Vector{Float64}, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024]) with eltype Float64:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [ ]:
# Simulate to check that magnitude is same as BlochSim.jl, and get target pattern to recover the pulse
Mx = zeros(N * os, 1)
My = zeros(N * os, 1)
Mz = zeros(N * os, 1)
Mx0 = 0
My0 = 0
Mz0 = 1.0
for ii = 1 : N * os
    rfg = [rf * dt / 1000 * GAMMA; b1[ii]; 2 * π * 0 / N; Mx0; My0; Mz0];
    M = myBlochSimB1(rfg)
    Mx[ii] = M[1]
    My[ii] = M[2]
    Mz[ii] = M[3]
end
plot(abs.(Complex.(Mx, My)))

In [30]:
plot(Mxd)
plot!(Myd)
plot!(Mzd)
plot!(w)

<!DOCTYPE html>
 
 
 Plots.jl

In [ ]:
#rf = rf * 0
rf = rf ./ rf
step = 0.0001
rfocinv = 4 * rf * GAMMA * dt / 1000 / 1000
niters = 1000
g = b1 -> ForwardDiff.gradient(myBlochSimErr, b1)#optimization gradient
#h = x -> ForwardDiff.hessian(myBlochSimErr, x)
Mx0 = 0
My0 = 0
Mz0 = 1.0
@showprogress 1 "Computing..." for nn = 1 : niters
    J = zeros(N, 1)
    #H = zeros(nrf, nrf)
    for ii = 1 : N * os
        rfg = [rfocinv; b1[ii]; Mx0; My0; Mz0; Mxd[ii]; Myd[ii]; Mzd[ii]; w[ii]]
        J += g(rfg)[1 : end - 8] 
        #rfg = [rfocinv; b1[ii]; B12wrf(b1[ii],5000)/wrf[end] * 2 * π; Mx0; My0; Mz0; 
            #Mxd[ii]; Myd[ii]; Mzd[ii]; w[ii]]
        #J += g(rfg)[1 : end - 9] 
        #H += h(rfg)[1 : end - 4, 1 : end - 4]
    end
    #rfoc -= (H \ J) 
    rfocinv -= step * J
end
plot(rfocinv)

Computing...  0%|█                                      |  ETA: 7:43:23

In [ ]:
display(plot(g(b1)))
test_mx,test_my,test_mz = myBlochSimB1([rf ;b1[floor(Int,nb1/2)] ;B12wrf(b1[floor(Int,nb1/2)],5000)/wrf[end] * 2 * π; 0; 0; 1.0])
#display(plot(test_mx))

In [ ]:
# Simulate result - looks great!
Mx = zeros(N * os, 1)
My = zeros(N * os, 1)
Mz = zeros(N * os, 1)
Mx0 = 0
My0 = 0
Mz0 = 1.0
for ii = 1 : N * os
    rfg = [rfocinv;b1[ii]; B12wrf(b1[ii],5000)/wrf[end] * 2 * π; Mx0; My0; Mz0];
    rfg_scale = rfg
    M = myBlochSimB1(rfg_scale)
    Mx[ii] = M[1]
    My[ii] = M[2]
    Mz[ii] = M[3]
end
plot(Mz)

In [ ]:
# simulate a b1-selective pulse that we designed previously in MATLAB

In [ ]:
# now let's try a Newton-based step
step = 1
rfocinv = rf * GAMMA * dt / 1000 / 1000
niters = 4
g = x -> ForwardDiff.gradient(myBlochSimErr, x)
h = x -> ForwardDiff.hessian(myBlochSimErr, x)
@showprogress 1 "Computing..." for nn = 1 : niters
    J = zeros(N, 1)
    H = zeros(N, N)
    for ii = 1 : N * os
        rfg = [rfocinv; 2 * π * x[ii] / N; Mxd[ii]; Myd[ii]; Mzd[ii]; w[ii]]
        J += g(rfg)[1 : end - 5] 
        H += h(rfg)[1 : end - 5, 1 : end - 5]
    end
    rfocinv -= step * (H \ J)
    #rfocinv -= step * J

end
plot(rfocinv)

In [ ]:
# do a spin echo pulse design

# target pattern parameters
tb = 8
d1 = 0.01
d2 = 0.01
ftw = dinf(d1 / 4.0, sqrt(d2)) / tb # spin echo transition width relationship

N = 128
f = [0, (1 - ftw) * (tb / 2), (1 + ftw) * (tb / 2), (N / 2)] / (N / 2)
os = 8
x = (-N / 2 : 1 / os : N / 2 - 1 / os)
xx = [x; x]

# set up initial conditions - we will use two ICs, to ensure complex conjugation. See Fig 8 in Conolly optimal control
Mx01 = zeros(N * os, 1)
Mx01[abs.(collect(x) ./ (N / 2)) .< f[2]] .= 1.0
My01 = zeros(N * os, 1)
Mz01 = ones(N * os, 1)
Mz01[abs.(collect(x) ./ (N / 2)) .< f[2]] .= 0.0

Mx02 = zeros(N * os, 1)
My02 = zeros(N * os, 1)
My02[abs.(collect(x) ./ (N / 2)) .< f[2]] .= 1.0
Mz02 = ones(N * os, 1)
Mz02[abs.(collect(x) ./ (N / 2)) .< f[2]] .= 0.0

Mx0 = [Mx01; Mx02]
My0 = [My01; My02]
Mz0 = [Mz01; Mz02]
#Mx0 = 0.0
#My0 = 1.0
#Mz0 = 0.0

# set up the target patterns - we want to leave Mx alone when M0 = [1, 0, 0], and negate My when M0 = [0, 1, 0] 
Mxd1 = Mx01
Myd1 = My01

Mxd2 = zeros(N * os, 1)
Myd2 = -copy(My02)

Mxd = [Mxd1; Mxd2]
Myd = [Myd1; Myd2]
Mzd = copy(Mz0)

# error weights
w = zeros(Float64, N * os, 1)
w[abs.(collect(x) ./ (N / 2)) .< f[2]] .= 1.0
w[abs.(collect(x) ./ (N / 2)) .> f[3]] .= d1 / d2
#w2 = zeros(N * os, 1)
#w2[abs.(collect(x) ./ (N / 2)) .< f[2]] .= d1 / d2
#w2[abs.(collect(x) ./ (N / 2)) .> f[3]] .= 1.0
w = [w; w]

In [ ]:
plot(Mxd, label="Desired Mx")
plot!(Myd, label="Desired My")
plot!(Mzd, label="Desired Mz")
plot!(w, label="weights")
plot!(xx ./ N, label="space")

In [ ]:
step = 0.0001
rfocref = 4 * rf * GAMMA * dt / 1000
niters = 1000
g = x -> ForwardDiff.gradient(myBlochSimErr, x)
@showprogress 1 "Computing..." for nn = 1 : niters
    J = zeros(N, 1)
    for ii = 1 : length(Mxd)
        rfg = [rfocref; 2 * π * xx[ii] / N; Mx0[ii]; My0[ii]; Mz0[ii]; Mxd[ii]; Myd[ii]; Mzd[ii]; w[ii]]
        J += g(rfg)[1 : end - 8] 
    end
    rfocref -= step * J
end
plot(rfocref)

In [ ]:
# Simulate result
Mx1 = zeros(N * os, 1)
My1 = zeros(N * os, 1)
Mz1 = zeros(N * os, 1)
for ii = 1 : N * os
    rfg = [rfocref; 2 * π * x[ii] / N; Mx0[ii]; My0[ii]; Mz0[ii]];
    M = myBlochSim(rfg)
    Mx1[ii] = M[1]
    My1[ii] = M[2]
    Mz1[ii] = M[3]
end
plot(Mx1, label="Mx")

Mx2 = zeros(N * os, 1)
My2 = zeros(N * os, 1)
Mz2 = zeros(N * os, 1)
for ii = 1 : N * os
    rfg = [rfocref; 2 * π * x[ii] / N; Mx0[ii + N * os]; My0[ii + N * os]; Mz0[ii + N * os]];
    M = myBlochSim(rfg)
    Mx2[ii] = M[1]
    My2[ii] = M[2]
    Mz2[ii] = M[3]
end
plot!(My2, label="My")
plot!(w[1 : N * os], label="w")